<a href="https://colab.research.google.com/github/vodkolav/DSlab1/blob/Colab/DSlab2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2 submission
for course: Data Science lab 2   
by  Michael Berger   
 

Lecturer: Dr. Sharon Yalov-Handzel  
Afeka College of engineering


In [ ]:
context = "DEBUG"
#context = "RELEASE"

try:
  from google.colab import drive
  drive.mount('/content/drive',)
  pfx = "/content/drive/MyDrive/Studies/M.Sc/DSlab2"
  IN_COLAB = True
except:
  pfx = ""
  IN_COLAB = False

## 1. Use the dataset from UCI Machine Learning Repository:
"Individual household electric power consumption" for performing time series analysis.  
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

In [ ]:
! pip install ucimlrepo
! pip install scikit-learn==1.4.2
! pip install -U kaleido

In [ ]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, clone_model
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.utils import plot_model

import os
from datetime import datetime
import statistics as stat

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

from ucimlrepo import fetch_ucirepo , dotdict
pd.set_option('display.precision', 3)

from pathlib import Path
import pickle 


In [ ]:
run_id = datetime.now().strftime("%Y.%m.%d_%H-%M-%S")
run_id

In [ ]:
# make plotly plots appear on github preview
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "notebook+pdf"
#pio.renderers.default = "sphinx_gallery"
#pio.renderers.default = "plotly_mimetype+jpeg"

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
fn = r"data/individual_household_electric_power_consumption.pkl"
pth = Path(pfx) / Path(fn)

In [ ]:
if pth.exists():
    with open(pth, "rb") as fl:
        individual_household_electric_power_consumption = pickle.load(fl)
else: 
    
    individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
    pth.parent.mkdir(parents=True, exist_ok=True)
    with open(pth, "wb") as fl:
        pickle.dump(individual_household_electric_power_consumption, fl)
    

In [ ]:
# data (as pandas dataframes) 
X = individual_household_electric_power_consumption.data.features 
y = individual_household_electric_power_consumption.data.targets 
  
# variable information 
print(individual_household_electric_power_consumption.variables) 

In [ ]:
import json
# metadata 
meta = json.dumps(individual_household_electric_power_consumption.metadata,indent=2)
print(meta)

In [ ]:
metadic = individual_household_electric_power_consumption.metadata

In [ ]:
def mylen(o):
    if type(o)==int:
        return str(o)
    if type(o)==type(None):
        return "0"
    else:
        return len(o)

In [ ]:
def showkeys(dic, ind = 0):
    if type(dic)== dotdict:
        for k,v  in dic.items() :
            print(" "*ind , k, "|", type(v).__name__, ",", mylen(v) )
            showkeys(dic[k], ind+4)

showkeys(metadic)

In [ ]:
print(metadic["abstract"])

In [ ]:
print(metadic["additional_info"]["summary"])

In [ ]:
print(metadic["additional_info"]["variable_info"])

## 2. Perform Exploratory Data Analysis (EDA) of the dataset:

First, convert all values to numeric

In [ ]:
numcols = ['Global_active_power', 'Global_reactive_power',
           'Voltage', 'Global_intensity', 'Sub_metering_1', 
           'Sub_metering_2', 'Sub_metering_3']

for c in numcols:
    X[c] = pd.to_numeric(X[c], errors = "coerce") 

combine date and time into single col datetime

In [ ]:
dt = X["Date"] + " " + X["Time"]
X["datetime"] = pd.to_datetime(dt, dayfirst=True)
X = X.set_index('datetime').drop(["Date","Time"],axis=1)

### a.  Visualize time series trends

In [ ]:
def summary(df):
    nonnans = df.shape[0] - df.isna().sum()
    nonnansPrc = (nonnans / df.shape[0] * 100).apply("{0:.2f}%".format)
    sam = df.sample(1, random_state=42).squeeze()
    res = pd.DataFrame([sam.index, df.dtypes.astype(str), nonnans,
                        nonnansPrc, df.nunique(), sam]).transpose()
    res.columns = ["Column", "data type", "non-null values", 
                   "non-null values %", "unique values", "example"]
    res.sort_values("unique values",ascending=False, inplace=True)
    return res

In [ ]:
X.head()

In [ ]:

pd.options.display.float_format = '{:,.4f}'.format
d = X.describe()
d

In [ ]:
summary(X)

### b. Check for seasonality and cyclical patterns

- Analyze distribution of power consumption
- Identify and handle missing values or outliers

## 3. Implement a linear regression model to predict power consumption for the last three time periods:

extract a smaller set Xs for debugging  

In [ ]:
# backup original dataset as data to re-process later
data = X.copy(deep=True)


# debug and development values: take small chunk of data,
# few epochs and verbose output
if context == "DEBUG":
  n_sampls = 10000
  X = X.iloc[0:n_sampls,:]
  time_step = 12
  epochs = 2
  batch_size = 64
  verbose = 1
else:
  print("running RELEASE config. Grab a drink an a snack, it's gonna take long.")    
  time_step = 12
  epochs = 15
  batch_size = 1024
  verbose = 1

### b. Prepare features (consider lag variables, time-based features) 
features: 
year
month 
week
day
hour
every variable + 10 of it's lags 

y's: 
Global_active_power

In [ ]:
# resample the data to fill the NaN values
#   Xs  or X
df= X["Global_active_power"].resample("1min").mean().ffill().to_frame()
df

In [ ]:
# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
tmp = scaler.fit_transform(df)
df["Global_active_power"] = tmp
df

In [ ]:
for i in range(1,time_step):
    lag = i
    df[f"Global_active_power.L{lag}"] = df["Global_active_power"].shift(lag)

df.dropna(inplace = True)
df

### a. Split the data into training and testing sets

In [ ]:
# find appropriate train/test ratio for the data
td = df.index.max() - df.index.min()
td.days * 24 * 60

In [ ]:
td.days / 30

In [ ]:
X.shape

In [ ]:
1- (3 * 30 * 24 * 60 ) / X.shape[0] 

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
t = df[target]
P = df.drop(columns= target)                                             
#Split the data into train and test sets
cutoff = int(len(P) * 0.94) # size of train
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

In [ ]:
X_train.shape, X_test.shape

### c. Train the model and make predictions

In [ ]:
LR_model = LinearRegression()
res = LR_model.fit(X_train, y_train)


In [ ]:
df_pred_test = pd.DataFrame(
             {
             "y_test": y_test,
             "LinearRegression": LR_model.predict(X_test).squeeze()
             })
df_pred_test

In [ ]:
def compare_preds(df):
    # compare predictions of different models 
    # over a small (for performance) chunk of data 
    df_pred_test = df[:10000]

    bkp = Path(pfx) / Path("backup") / Path(run_id)
    bkp.mkdir(parents=True, exist_ok=True)
    df_pred_test.to_json(bkp / Path("test_preds.json"), indent=4)
    
    fig = px.line(df_pred_test, labels= {"value": "Powah!"}, 
              title = "Actual powah vs. Predicted powah.")
    return fig

In [ ]:
compare_preds(df_pred_test)

## 4. Evaluate the linear regression model using appropriate metrics:
- Mean Absolute Error (MAE)
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- R-squared (R²) value

In [ ]:
experiments = pd.DataFrame(columns=['model','data'])

In [ ]:
def sqiz(y):
    # required to deal with higher dimensional outputs 
    if len(y.shape)>2 and y.shape[2]>1:
        y = y[:,0,0]
    return y.squeeze()


def pipeline(model, x_train, y_train, x_test, y_test, model_args={}):
    
    margs = model_args.copy() # there is no pass-by-value in python, duh    
    # Train the model
    print(f"training model, please wait...")
    model.fit(x_train, y_train, **margs) # , epochs=epochs, validation_split= .1, verbose=0)
    
    if len(model_args.values())>0:
        margs.pop("epochs")
        margs.pop("verbose")

    print(f"evaluating model, please wait...")
    y_trn_pred = model.predict(x_train, **margs)    
    y_pred = model.predict(x_test, **margs)

    #TODO: Invert predictions back to original values range?
    # train_predict = scaler.inverse_transform(train_predict)
    # Y = scaler.inverse_transform([Y])
    
    #df_pred_test = pd.DataFrame({"y_test": y_test,"y_pred": y_pred})
   
    res = {}
    print("1) y_train.shape: ", y_train.shape,"| y_trn_pred.shape: ", y_trn_pred.shape)
    # Evaluate the model on train set
    y_train = sqiz(y_train)
    y_trn_pred = sqiz(y_trn_pred)
    print("2) y_train.shape: ", y_train.shape,"| y_trn_pred.shape: ", y_trn_pred.shape)
    
    # note: [] on rhs are required for converting to df later on
    res["trn_MAE"] = [mean_absolute_error(y_train, y_trn_pred)]
    res["trn_MSE"] = [mean_squared_error(y_train, y_trn_pred)]
    res["trn_RMSE"] = [root_mean_squared_error(y_train, y_trn_pred)]
    res["trn_Rsq"] = [r2_score(y_train, y_trn_pred)]

    print("1) y_test.shape: ", y_test.shape,"| y_pred.shape: ", y_pred.shape)
    # Evaluate the model on test set
    y_test = sqiz(y_test)
    y_pred = sqiz(y_pred)
    print("2) y_test.shape: ", y_test.shape,"| y_pred.shape: ", y_pred.shape)
    
    res["tst_MAE"] = [mean_absolute_error(y_test, y_pred)]
    res["tst_MSE"] = [mean_squared_error(y_test, y_pred)]
    res["tst_RMSE"] = [root_mean_squared_error(y_test, y_pred)]
    res["tst_Rsq"] = [r2_score(y_test, y_pred)]
    print("done")
    return model, res
  

In [ ]:

def report(result, model_label, data_label):
    a = result
    a["model"] = model_label
    a["data"] = data_label
    log_df = pd.DataFrame(a)    
    global experiments 

    row = experiments.query(f"model=='{model_label}' and data == '{data_label}'")    
    if(experiments.shape[0]>0) and len(row) > 0:
        import warnings
        warnings.warn(f"experiment data  for model label '{model_label}' \n\
        and data label '{data_label}' already exist. replacing.")        
        experiments.drop(row.index, inplace=True)
        #experiments[["model","data"]].head()
        #drop(fit_runs, 'data', label)
    
    experiments = pd.concat([experiments, log_df])
    experiments.reset_index(drop=True, inplace=True)

    bkp = Path(pfx) / Path("backup") / Path(run_id)
    bkp.mkdir(parents=True, exist_ok=True)
    experiments.to_json(bkp / Path("experiments.json"), indent=4)
   
    #print(experiments.head())

    metrics = list(result.keys())
    metrics.sort()
    
    log_df_melted = experiments.melt(id_vars=['model','data'], 
                            value_vars= metrics, #['accuracy', 'loss'], 
                            var_name='metric', 
                            value_name='value')
    
    rx = r"(?P<set>[trns]+_)?(?P<metric>[\dA-Za-z-]+)"
    nspl = log_df_melted.metric.str.extract(rx)
    log_df_melted.drop('metric',axis=1, inplace=True)
    log_df_melted = log_df_melted.join(nspl)    
    log_df_melted.loc[log_df_melted.set == "trn_", "set"] = "trn"
    #log_df_melted.loc[log_df_melted.set == "val_", "set"] = "val"
    log_df_melted.loc[log_df_melted.set == "tst_", "set"] = "tst"

    log_df_melted = log_df_melted.query("metric!='Rsq'")  


    #vertical bars
    # dims = {'x':'data', 
    #         'y':'value', 
    #         'facet_row':'metric',
    #         'facet_col':'model',   
    #         'color':'set'}

    #horizontal bars
    dims = {'x':'value', 
            'y':'data', 
            'facet_row':'model',
            'facet_col':'metric',   
            'color':'set'}

    ttl = "table cols: metrics| cell x axis: value of given metric for given dataset and model  <br>" + \
          "table rows: models | cell y axis: different processings of the dataset | colors: training or testing set<br>"
    
    fig = px.bar(log_df_melted, title= ttl,
                 barmode = 'group', text_auto='.2',
                 category_orders={"set": ["trn","val","tst"]},
                 **dims )
   
    #fig.update_yaxes(matches=None,)
    # Update layout
    fig.update_layout(height=800, width=1000)
    fig.update_traces(textfont_size=14, textangle=0, textposition="outside", cliponaxis=True)

    #fig.update_yaxes(matches='y domain')
    return fig

In [ ]:
baseline_model = DummyRegressor(strategy='mean')

In [ ]:
baseline_model, res = pipeline(baseline_model, X_train, y_train, X_test, y_test)

report(res, "Baseline", "basic")


In [ ]:
df_pred_test["baseline"] =  baseline_model.predict(X_test)

In [ ]:
model = LinearRegression()

In [ ]:
model, res = pipeline(model, X_train, y_train, X_test, y_test)
report(res, "LinearRegression", "basic")

In [ ]:
compare_preds(df_pred_test)

In [ ]:
experiments

## 5. Recurrent Neural Network (RNN)
Implement a Recurrent Neural Network (RNN) for power consumption prediction:

### a. Preprocess data for RNN input

In [ ]:
df.shape

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
t = df[target]
P = df.drop(columns= target)                     


#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

### b. Design and train the RNN model

In [ ]:
#sequence_length = X_train.shape[1]

# Build an RNN model
Rnn_model = Sequential([
    Input((time_step, 1)),
    SimpleRNN(units=32, activation='relu'),
    Dense(units=1)
])

Rnn_model.compile(optimizer='adam', loss='mse')
model_args = {"epochs":epochs, "batch_size":batch_size, "verbose": verbose}

In [ ]:
Rnn_model, res = pipeline(Rnn_model, X_train, y_train, X_test, y_test, model_args)

### c. Make predictions and visualize results  

 TODO: visualise the time series with predictions 

In [ ]:
df_pred_test["RNN"] =  Rnn_model.predict(X_test)

In [ ]:
compare_preds(df_pred_test)

### d. Compare performance metrics with linear regression

In [ ]:
report(res, "RNN", "basic")

## 6. Long Short-Term Memory (LSTM)
Implement Long Short-Term Memory (LSTM) for power consumption prediction:
 

### a. Preprocess data for LSTM input

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
t = df[[target]]
P = df.drop(columns= target)                     

#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

In [ ]:
X_train.shape, y_train.shape

### b. Design and train the LSTM model

In [ ]:
# Create and fit the LSTM network
LSTM_model = Sequential()
LSTM_model.add(Input((time_step, 1)))
LSTM_model.add(LSTM(50, return_sequences=True ))
LSTM_model.add(LSTM(50, return_sequences=False))
LSTM_model.add(Dense(1))

LSTM_model.compile(optimizer='adam', loss='mean_squared_error')

model_args = {"epochs":epochs, "batch_size":batch_size, "verbose":verbose}

In [ ]:
plot_model(LSTM_model ,
    show_shapes=True,
    #show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=200,
    show_layer_activations=False,
    show_trainable=True)

In [ ]:
model_args

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
LSTM_model, res = pipeline(LSTM_model, X_train, y_train, X_test, y_test, model_args)

### c. Make predictions and visualize results

In [ ]:
df_pred_test["LSTM"] =  LSTM_model.predict(X_test)

In [ ]:
compare_preds(df_pred_test)

### d. Compare performance metrics with previous models

In [ ]:
report(res, "LSTM", "basic")

## 7. LSTM with Attention
Implement an LSTM model with an Attention layer for power consumption prediction:

### a. Design and train the LSTM model with Attention

In [ ]:
# initializing context length, goes to decoder
d = 4
# the rest is historical  context, goes to encoder

t    = df.iloc[:,0:d]
Xdec = df.iloc[:,1:(d+1)]
Xenc = df.iloc[:,(d+1):]

[Xenc.shape, Xdec.shape], t.shape 

In [ ]:
#Split the data into train and test sets
X_train, y_train = [Xenc[:cutoff], Xdec[:cutoff]], t[:cutoff]
X_test , y_test  = [Xenc[cutoff:], Xdec[cutoff:]], t[cutoff:]

In [ ]:
[X_train[0].shape, X_train[1].shape], y_train.shape

In [ ]:
# Attention-based Encoder-Decoder model

encoder_input = Input(shape=(Xenc.shape[1], 1), name= "encoder_input")
encoder_lstm = LSTM(32, return_sequences=True, name="encoder_lstm")(encoder_input)

decoder_input = Input(shape=(Xdec.shape[1], 1), name= "decoder_input")
decoder_lstm = LSTM(32, return_sequences=True, name="decoder_lstm")(decoder_input)

attention = Attention(name = "attention")([decoder_lstm, encoder_lstm])
combined = Concatenate(axis=-1)([decoder_lstm, attention])

output = Dense(1)(combined)

attention_model = Model(inputs=[encoder_input, decoder_input], outputs=output)
attention_model.compile(optimizer='adam', loss='mse')


In [ ]:

plot_model(attention_model,
    show_shapes=True,
    #show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=200,
    show_layer_activations=False,
    show_trainable=True)

In [ ]:
attention_model, res = pipeline(attention_model,
                                X_train, y_train, 
                                X_test, y_test, 
                                model_args)

### b. Make predictions and visualize results

In [ ]:
att_pred =  attention_model.predict(X_test)

In [ ]:
df_pred_test["ATTN"] = att_pred[:,0]

In [ ]:
compare_preds(df_pred_test)

### c. Compare performance metrics with previous models

In [ ]:
report(res, "ATTN", "basic")

### d. Analyze the Attention weights to interpret model focus

## 9. Data reduction experiment:

### a. Remove up to 10% of the data randomly

In [ ]:
remove_n = int(len(df) * 0.10)
print(f"will remove {remove_n} rows")
drop_indices = np.random.choice(df.index, remove_n, replace=False)
dfm = df.drop(drop_indices)

cutoff = int(len(dfm) * 0.9) # size of train
dfm

In [ ]:

# Split the data into feature and target
target = "Global_active_power"
t = dfm[target]
P = dfm.drop(columns= target)        


In [ ]:
# DATA FOR lstm+Attn
# initializing context length, goes to decoder
d = 4
# the rest is historical  context, goes to encoder

tdec = dfm.iloc[:,0:d]
Xdec = dfm.iloc[:,1:(d+1)]
Xenc = dfm.iloc[:,(d+1):]

[Xenc.shape, Xdec.shape], t.shape 

In [ ]:
#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]


In [ ]:
dfm.shape, X_train.shape, y_train.shape

### b. Retrain and evaluate all three models

In [ ]:

baseline_model = DummyRegressor(strategy='mean')
baseline_model, baseline_res = pipeline(baseline_model, X_train, y_train, X_test, y_test)


In [ ]:

LR_model = LinearRegression()
LR_model, LR_res = pipeline(LR_model, X_train, y_train, X_test, y_test)


In [ ]:
# playing around
model_args = {"epochs":epochs, "batch_size":256, "verbose":verbose}
# margs = model_args.copy()
# model_args.pop("batch_size")

# print(asd)
print(model_args)
      

In [ ]:

Rnn_model = clone_model(Rnn_model)
Rnn_model.compile(optimizer='adam', loss='mse')
Rnn_model, Rnn_res = pipeline(Rnn_model, X_train, y_train, X_test, y_test, model_args)


In [ ]:

LSTM_model = clone_model(LSTM_model)
LSTM_model.compile(optimizer='adam', loss='mean_squared_error')
LSTM_model, LSTM_res = pipeline(LSTM_model, X_train, y_train, X_test, y_test, model_args)


In [ ]:
#Split the data into train and test sets
X_train, y_train = [Xenc[:cutoff], Xdec[:cutoff]], tdec[:cutoff]
X_test , y_test  = [Xenc[cutoff:], Xdec[cutoff:]], tdec[cutoff:]

In [ ]:

attention_model = clone_model(attention_model)
attention_model.compile(optimizer='adam', loss='mse')
attention_model, attn_res = pipeline(attention_model, 
                                X_train, y_train, 
                                X_test, y_test, model_args)


### c. Compare how data reduction affects each model's performance

In [ ]:
report(baseline_res, "Baseline", "reduction")
report(LR_res, "LinearRegression", "reduction")
report(Rnn_res, "RNN", "reduction")
report(LSTM_res, "LSTM", "reduction")
report(attn_res, "ATTN", "reduction")

## 10. Data resolution experiment:

### a. Reduce the time resolution of the data by 50% (e.g., from minute-level to 2-minute intervals)

In [ ]:
taake = range(0, len(df), 2)
dfm = df.iloc[taake]
cutoff = int(len(dfm) * 0.9) # size of train

In [ ]:

# Split the data into feature and target
target = "Global_active_power"
t = dfm[target]
P = dfm.drop(columns= target)        


In [ ]:
# DATA FOR lstm+Attn
# initializing context length, goes to decoder
d = 4
# the rest is historical  context, goes to encoder

tdec = dfm.iloc[:,0:d]
Xdec = dfm.iloc[:,1:(d+1)]
Xenc = dfm.iloc[:,(d+1):]

[Xenc.shape, Xdec.shape], t.shape 

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
t = dfm[target]
P = dfm.drop(columns= target)        

In [ ]:
#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

In [ ]:
df.shape, X_train.shape, y_train.shape

### b. Retrain and evaluate all three models

In [ ]:

baseline_model = DummyRegressor(strategy='mean')
baseline_model, baseline_res = pipeline(baseline_model, X_train, y_train, X_test, y_test)


In [ ]:

LR_model = LinearRegression()
LR_model, LR_res = pipeline(LR_model, X_train, y_train, X_test, y_test)


In [ ]:

Rnn_model = clone_model(Rnn_model)
Rnn_model.compile(optimizer='adam', loss='mse')
Rnn_model, Rnn_res = pipeline(Rnn_model, X_train, y_train, X_test, y_test, model_args)


In [ ]:

LSTM_model = clone_model(LSTM_model)
LSTM_model.compile(optimizer='adam', loss='mse')
LSTM_model, LSTM_res = pipeline(LSTM_model, X_train, y_train, X_test, y_test, model_args)


In [ ]:
#Split the data into train and test sets
X_train, y_train = [Xenc[:cutoff], Xdec[:cutoff]], tdec[:cutoff]
X_test , y_test  = [Xenc[cutoff:], Xdec[cutoff:]], tdec[cutoff:]

In [ ]:

attention_model = clone_model(attention_model)
attention_model.compile(optimizer='adam', loss='mse')
attention_model, attn_res = pipeline(attention_model, 
                                X_train, y_train, 
                                X_test, y_test, model_args)


### c. Analyze how changes in data resolution impact each model's performance

In [ ]:
report(baseline_res, "Baseline", "resolution")
report(LR_res, "LinearRegression", "resolution")
report(Rnn_res, "RNN", "resolution")
report(LSTM_res, "LSTM", "resolution")
report(attn_res, "ATTN", "resolution")

## 8. Data augmentation experiment:

### a. Modify up to 10% of the dataset to potentially improve prediction results
I did not understand how I should modify the dataset to improve results, so I just added another variable to the mix.

In [ ]:
def preprocess(X, colname,  time_step = 12):
    df = X[colname].resample("1min").mean().ffill().to_frame()
    # Normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    tmp = scaler.fit_transform(df)
    df[colname] = tmp   
    
    for lag in range(1,time_step):
        df[f"{colname}.L{lag}"] = df[colname].shift(lag)
    
    df.dropna(inplace = True)
    return df

gap = preprocess(X, "Global_active_power")
grp = preprocess(X, "Global_reactive_power")
df = gap.join(grp)

### b. Retrain and evaluate all three models (RNN, LSTM, LSTM with Attention)

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
t = df[target]
P = df.drop(columns= target)

#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

#### Classic

In [ ]:

baseline_model = DummyRegressor(strategy='mean')
baseline_model, baseline_res = pipeline(baseline_model, X_train, y_train, X_test, y_test)


In [ ]:

LR_model = LinearRegression()
LR_model, LR_res = pipeline(LR_model, X_train, y_train, X_test, y_test)


#### RNN

In [ ]:
#sequence_length = X_train.shape[1]

# Build an RNN model
Rnn_model = Sequential([
    Input((X_train.shape[1], 1)),
    SimpleRNN(units=32, activation='relu'),
    Dense(units=1)
])

Rnn_model.compile(optimizer='adam', loss='mse')


In [ ]:
model_args = {"epochs":epochs, "batch_size":batch_size, "verbose": verbose}

In [ ]:
Rnn_model, RNN_res = pipeline(Rnn_model, X_train, y_train, X_test, y_test, model_args)

#### LSTM  
Requires slightly different preprocessing:

In [ ]:
gap = np.expand_dims(gap.values, axis=2)
grp = np.expand_dims(grp.values, axis=2) 
arr = np.concatenate([gap, grp], axis=2)

In [ ]:
# Split the data into feature and target
t = arr[:,0,:]
P = arr[:,1:,:]
#Split the data into train and test sets
X_train, y_train = P[:cutoff], t[:cutoff]
X_test, y_test = P[cutoff:], t[cutoff:]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
input_dim = X_train.shape[1:3]
input_dim

In [ ]:
# Create and fit the LSTM network
LSTM_model = Sequential()
LSTM_model.add(Input(input_dim))
LSTM_model.add(LSTM(50, return_sequences=True ))
LSTM_model.add(LSTM(50, return_sequences=False))
LSTM_model.add(Dense(input_dim[1]))

LSTM_model.compile(optimizer='adam', loss='mean_squared_error')

model_args = {"epochs":epochs, "batch_size":32, "verbose":verbose}

In [ ]:
LSTM_model, LSTM_res = pipeline(LSTM_model, X_train, y_train, X_test, y_test, model_args)

#### LSTM with attention

In [ ]:
# initial context length, goes to decoder
d = 4
# the rest is historical  context, goes to encoder

t    = arr[:,0:d]
Xdec = arr[:,1:(d+1)]
Xenc = arr[:,(d+1):]

t.shape, Xdec.shape, Xenc.shape

In [ ]:

#Split the data into train and test sets

X_train, y_train = [Xenc[:cutoff], Xdec[:cutoff]], t[:cutoff]
X_test , y_test  = [Xenc[cutoff:], Xdec[cutoff:]], t[cutoff:]

In [ ]:
[X_train[0].shape, X_train[1].shape], y_train.shape

In [ ]:
[X_test[0].shape, X_test[1].shape], y_test.shape

In [ ]:
# Attention-based Encoder-Decoder model

encoder_input = Input(shape=Xenc.shape[1:3], name= "encoder_input")
encoder_lstm = LSTM(32, return_sequences=True, name="encoder_lstm")(encoder_input)

decoder_input = Input(shape=Xdec.shape[1:3], name= "decoder_input")
decoder_lstm = LSTM(32, return_sequences=True, name="decoder_lstm")(decoder_input)

attention = Attention(name = "attention")([decoder_lstm, encoder_lstm])
combined = Concatenate(axis=-1)([decoder_lstm, attention])

output = Dense(Xenc.shape[2])(combined)

attention_model = Model(inputs=[encoder_input, decoder_input], outputs=output)
attention_model.compile(optimizer='adam', loss='mse')
#, metrics= <keras.metrics.>[mean_squared_error, 
# RootMeanSquaredError, mean_absolute_error, R2Score])


In [ ]:
plot_model(attention_model ,
    show_shapes=True,
    #show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=200,
    show_layer_activations=False,
    show_trainable=True)

In [ ]:
attention_model, attention_res = pipeline(attention_model,
                                X_train, y_train, 
                                X_test, y_test, 
                                model_args)

### c. Compare the impact of data changes on each model's performance
  

In [ ]:
report(baseline_res, "Baseline", "augmented")
report(LR_res, "LinearRegression", "augmented")
report(RNN_res,  "RNN",  "augmented")
report(LSTM_res, "LSTM", "augmented")
report(attention_res, "ATTN", "augmented")

In [ ]:
print("run", run_id, "complete.")

## 11. Conclusion and insights:

- Summarize findings from all experiments
- Discuss which model performed best under different conditions
- Provide insights on the dataset's characteristics and their impact on model performance